<a href="https://colab.research.google.com/github/Gosuzkosmosu/SDA_projekt/blob/main/NLP_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq transformers==4.28.0
!pip -qq install evaluate
!pip install -qq datasets

##Importy

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer
from datasets import load_metric, Dataset

##Wczytanie danych

In [ ]:
# Wczytanie danych
df = pd.read_csv("z_tekstem_cudzyslowy.csv", on_bad_lines='skip')

In [ ]:
# Przetwarzanie danych
df['tekst'] = df['tekst'].apply(lambda x: str(x))

def split_into_chunks(text, max_chunk_length=510):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    for word in words:
        if current_length + len(word) + 1 > max_chunk_length:  # +1 dla wolnego miejsca
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word)
        else:
            current_chunk.append(word)
            current_length += len(word) + 1  # +1 dla wolnego miejsca
    chunks.append(" ".join(current_chunk))  # Dodawanie ostatniego chunk
    return chunks

# Zastosowanie chunkowania
df['tekst_chunks'] = df['tekst'].apply(split_into_chunks)

# Spłaszczenie dataset
flat_texts = []
flat_labels = []
for _, row in df.iterrows():
    for chunk in row['tekst_chunks']:
        flat_texts.append(chunk)
        flat_labels.append(row['Rodzaj nieruchomosci'])

##Label encoder

In [ ]:
label_encoder = preprocessing.LabelEncoder()
flat_labels_encoded = label_encoder.fit_transform(flat_labels)

In [ ]:
df

,Rodzaj nieruchomosci,Numer sieciowy,tekst,tekst_chunks
0,zabudowana,11OO_9Q_434YB/0C8F,"""Akt not. 6930/2023 z dnia 2023-11-23\n12\nnum...","[""Akt not. 6930/2023 z dnia 2023-11-23 12 nume..."
1,zabudowana,11OO_A6_318L8/0C8F,"""Akt not. 3291/2023 z dnia 2023-10-30\n\nb) Ni...","[""Akt not. 3291/2023 z dnia 2023-10-30 b) Nier..."
2,zabudowana,11OO_A5_3ES9O/0C8F,"""Akt not. 6882/2023 z dnia 2023-10-17\n\np\n\n...","[""Akt not. 6882/2023 z dnia 2023-10-17 p 4 WYP..."
3,zabudowana,11OO_9H_DZN3W/0C8F,"""Akt not. 67/2023 z dnia 2023\n\n»\n\n+0 wiAni...","[""Akt not. 67/2023 z dnia 2023 » +0 wiAniaóż S..."
4,zabudowana,11OO_9G_4IZHE/0C8F,"""dnia 2023-09-27\n\nLeasing — Eko Profit Spółk...","[""dnia 2023-09-27 Leasing — Eko Profit Spółki ..."
...,...,...,...,...
3661,niezabudowana,UV0_AK_4XLKDM/0C8F,"""sj""\nWio Kodyski niabrsatzpog sz)\n\nniyw ofo...","[""sj"" Wio Kodyski niabrsatzpog sz) niyw ofoj t..."
3662,niezabudowana,UV0_CT_5TCTF1/0C8F,"""SE\n\nWOD\nWOK\n\n== BJUGZOPEIMgO SIMEJSpOd B...","[""SE WOD WOK == BJUGZOPEIMgO SIMEJSpOd BU ISMO..."
3663,niezabudowana,UV0_CS_435VRO/0C8F,"""13200)\n\n'esatzpodiwelzp BI0łE\n\n+\nu\n\nu ...","[""13200) 'esatzpodiwelzp BI0łE + u u m (AZSO01..."
3664,niezabudowana,UV0_CP_3GBX1W/0C8F,"""(\n\n""T4E7\n\n6 00'00\n\nI* 00'09\n\nI* 00'00...","[""( ""T4E7 6 00'00 I* 00'09 I* 00'00z F 09'6bz'..."


##Splitowanie danych

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    flat_texts, flat_labels_encoded, test_size=0.2, stratify=flat_labels_encoded, random_state=42
)

## Inicjalizacja tokenizera

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/489k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

##Preprocesing danych

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

##Tokenizacja

In [ ]:
train_dataset = Dataset.from_dict({"text": train_texts, "labels": train_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "labels": test_labels})

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/76451 [00:00<?, ? examples/s]

Map:   0%|          | 0/19113 [00:00<?, ? examples/s]

## Funkcja do obliczania metryk

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = load_metric("f1")
    return {"f1": metric.compute(predictions=predictions, references=labels, average="macro")}

##Inicjalizacja modelu

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("dkleczek/bert-base-polish-cased-v1", num_labels=len(label_encoder.classes_))

pytorch_model.bin:   0%|          | 0.00/531M [00:00<?, ?B/s]

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification 

## Inicjalizacja argumentów treningowych

In [ ]:
training_args = TrainingArguments(
    output_dir="models",
    learning_rate=2e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    group_by_length=True,
)

## Inicjalizacja trenera

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

## Trening modelu

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,0.525500,0.488091,{'f1': 0.60317031884208}
2,0.467100,0.464990,{'f1': 0.6276657916928076}
3,0.447200,0.455888,{'f1': 0.6439853923901092}
4,0.409500,0.429109,{'f1': 0.675160834050265}
5,0.373300,0.420563,{'f1': 0.6919365032504047}
6,0.364100,0.414521,{'f1': 0.6942158598958051}
7,0.338100,0.408813,{'f1': 0.704620102903053}
8,0.325000,0.416128,{'f1': 0.7069443703924702}
9,0.315700,0.412743,{'f1': 0.7104029032549279}
10,0.310300,0.414404,{'f1': 0.7117484611252768}


<ipython-input-14-6c9b405969f1>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Trainer is attempting to log a value of "{'f1': 0.60317031884208}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Trainer is attempting to log a value of "{'f1': 0.6276657916928076}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/datasets/loa

TrainOutput(global_step=23900, training_loss=0.39184916189026137, metrics={'train_runtime': 18576.6519, 'train_samples_per_second': 41.154, 'train_steps_per_second': 1.287, 'total_flos': 2.0115283898723328e+17, 'train_loss': 0.39184916189026137, 'epoch': 10.0})

## Zapisywanie modelu do pliku

In [ ]:
import shutil
from transformers import AutoModel

# Załóżmy, że masz model `model` gotowy do zapisania
# model = AutoModel.for_pretrained('twoj_model')

# Save the model to a directory
#model.save_pretrained("model")

# Zip the saved model directory
#shutil.make_archive("model", 'zip', "model")

#print("Model directory zipped successfully.")

# Teraz, aby pobrać zipped model na lokalny dysk
from google.colab import files
files.download('model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>